In [ ]:
import os

api_key = ""
project = ""
region = ""
authorizationToken = f"{os.environ['TEST_ACTIVATION_TOKEN']}" if os.getenv('TEST_ACTIVATION_TOKEN') else ""

dataset_id = "basic_subclustering"
clusteringType = "community-detection"
cutoff = 0.75
encode_type = "text"
vector_fields = ["product_title_clip_vector_"]
n_clusters = 10




In [ ]:
# because we don't know how to set env variables
try:
    import os
    os.environ["WORKFLOW_ID"] = job_id
except Exception as e:
    print(e)
    pass

In [ ]:

## Instantiate client ###
from relevanceai import Client 
client = Client(token=authorizationToken)

## Checking valid vector field ###
for v in vector_fields:
  if not '_vector_'in v:
    raise ValueError(f"'{v}' is not a valid vector field")

!pip install -q -U RelevanceAI==2.7.6
ds = client.Dataset(dataset_id)
try:
  if clusteringType == 'community-detection':
    cluster_method  = "community_detection"
    ds.cluster(
        model=cluster_method,
        cluster_config={"threshold": cutoff},
        vector_fields=vector_fields
    )
  elif clusteringType == 'kmeans':
    if ds.shape[0] < 5000:
      from sklearn.cluster import KMeans
      cluster_method = 'kmeans'
      model = KMeans(n_clusters=n_clusters, random_state=42)
      alias = f"{cluster_method}-{n_clusters}"
      ds.cluster(
          model=model,
          cluster_config={"n_clusters": n_clusters, "random_state": 42},
          vector_fields=vector_fields,
          alias=alias
      )
    else:
      cluster_method = 'minibatchkmeans'
      from sklearn.cluster import MiniBatchKMeans
      cluster_method = 'kmeans'
      model = MiniBatchKMeans(n_clusters=n_clusters, random_state=42)
      alias = f"{cluster_method}-{n_clusters}"
      ds.batch_cluster(
          model=model,
          cluster_config={"n_clusters": n_clusters, "random_state": 42},
          vector_fields=vector_fields,
          alias=alias
      )
  elif clusteringType == "kmedoids":
    from sklearn_extra.cluster import KMedoids
    cluster_method = "kmedoids"
    model = KMedoids(n_clusters=n_clusters, random_state=42, init="k-medoids++")
    alias = f"{cluster_method}-{n_clusters}"
    ds.cluster(
        model=model,
        vector_fields=vector_fields,
        alias=alias
    )

except Exception as e:
  raise ValueError(f'{e}')

print("Finished clustering your data, you may close this window.")

